In [54]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import logging
import requests
import urllib.request
import json
import numpy as np
from tensorflow.keras.models import load_model
from gensim.models import FastText
from konlpy.tag import Okt
import boto3
import sys
import pandas as pd
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re

if sys.version_info[0] < 3:
    from io import StringIO # Python 2.x
else:
    from io import StringIO # Python 3.x

# get your credentials from environment variables
aws_id = ''
aws_secret = ''

client = boto3.client('s3', aws_access_key_id=aws_id,
        aws_secret_access_key=aws_secret)

bucket_name = 'snucsv'

object_key = 'Naver_Comments_All_true.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

# load data
train_df_true = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train_true = train_df_true[label_names].values

train_df_true['doc_len'] = train_df_true['Comments'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df_true['doc_len'].mean() + train_df_true['doc_len'].std()).astype(int)

object_key = 'Naver_Comments_All_fake.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

# load data
train_df_fake = pd.read_csv(StringIO(csv_string))
label_names = ["label"]
y_train_fake = train_df_fake[label_names].values

train_df_fake['doc_len'] = train_df_fake['Comments'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df_fake['doc_len'].mean() + train_df_fake['doc_len'].std()).astype(int)

In [55]:
train_df_true

,Comments,label,doc_len
0,"""이번에 홍준표 대통령안되면 이나라망한거니 이민간다""",1,5
1,"""홍준표 속과 겉이 똑같은 사람이라 남 속일줄 모르고 정직하고 정많고 인간적인 인물...",1,18
2,"""홍준표처럼 정치잘하면서 사람 좋은 정치인 못봤음. 홍준표는 명품임.""",1,8
3,"""또라이 아니야?? 순서가 안맞지 유승민 김무성 니들이 탄핵시켰으니 수사 받고 재판...",1,31
4,"""깜냥도 안되는 승민아 네가 대통령 되려고 하니 네가 하는 말이 앞뒤가 안맞지? 차...",1,16
...,...,...,...
4820,"""타다가 비용이 좀 비싸도 의자 열시트부터 에어컨 의자 위치까지 완전 쾌적하고 바로...",1,42
4821,"""기본요금거리 카드안받고욕하고:뭐하러택시타나여\n버스나탑시당~""",1,2
4822,"""불법인데 사실 불법으로 내몰리는 사람들 중 가족을 책임져야 하는 사람들이 많다는 ...",1,22
4823,"""사정 급한건알지만 그래도 한댠500만원 준다고\n에 속지마라\n한달500만원직장이...",1,8


In [56]:
train_df_fake

,Comments,label,doc_len
0,"""도대체 뭐가 타격이란건지 뻔한 결론인데 저게 백운규를 상대로한 심의위냐 누가봐도...",0,18
1,"""언제부터 수사심의회 결정대로 했냐.한동훈,이성윤 기억안나나,,,,,,참고사항이니 ...",0,7
2,"""이대로 끝난게 아니란거는 니들도 알건데""",0,5
3,"""정권교체해서 국난과 재앙에서 벗어나자.""",0,4
4,"""진짜딴건몰라도 부동산만큼은 역대최악 악질정권이다 국민이 호구가아님 판단할텐데 그렇...",0,19
...,...,...,...
4785,"""직접 국민의 도장받지 않는것은 위헌이다. 지들끼리 짜웅해서 의원을 새로 만들자는 ...",0,27
4786,"""최창렬이 저런 것도 교수가?.......대통령 선거도 표의 등가성을 따지자. 재앙...",0,20
4787,"""지금은 법과 상식이 안통하는 시기이니 보수들도 과격하게 투쟁합시다!! 나는 공산당...",0,11
4788,"""비례한국당 만든다며??""",0,2


In [57]:
train_df_all = pd.concat([train_df_true, train_df_fake])
train_df_all = train_df_all.sample(frac=1, random_state=1004).reset_index(drop=True)
train_df_all

,Comments,label,doc_len
0,"""독립유공자 후손지원, 청년수당, 여성안심택배, 올빼미 버스, 서울로 7017, 도...",1,26
1,"""문정부 집값 잡을 생각이 없구나~~~그래 서민들을 우롱했다~~당신들도 가진자들이다...",1,18
2,"""부정적으로만 보지말시구여 얼마나 살기가 개같으면 코로나인데도 불구하고 모이는지가 ...",0,21
3,"""ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 뭐? 한반도 운전자론? 중립외교? 양쪽에서 이득을 취해?...",0,17
4,"""성범죄는 그나마 개선된거죠 사기죄는 한결같이 제자리...사기 당한 사람들 보면 거...",1,41
...,...,...,...
9610,"""어이 기자양반 그럼 분단비용은 싸게 먹힙니까? 돈으로 추산 안되는 한반도 디스카운...",0,26
9611,"""주휴시간 폐지, 최저임금 차등적용, 세금, 4대보험 본인부담 하게 만들어라.""",0,9
9612,"""연동률 20프로로 낮추어라""",0,3
9613,"""헛다리에 헛발질 자한당 답다""",0,4


In [58]:
train_df = train_df_all

In [59]:
import pandas as pd
from konlpy.tag import Okt
import numpy as np
import boto3
import sys
from gensim.models import FastText
import os, csv, math, codecs
from tqdm import tqdm
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding, Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


MAX_NB_WORDS = 100000

# =========================================================================
# Vectorizer의 argument인 tokenizer에 KoNLPy의 pos 함수로 대체.
class MyTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        pos = self.tagger.pos(sent)
        clean_words = []  # 정제된 단어 리스트
        for word in pos:
            # word[1]은 품사를 의미하며, 여기서는 조사, 문장기호, 접두사, Foreign('\n'을 빼주기 위함)인 것은 제외시킴.
            if word[1] not in ['Josa', 'Punctuation', 'Suffix', 'Foreign']:
                if len(word[0]) >= 2:  # 한 글자인 단어들도 의미가 없는 경우가 많으므로 일단 제외.
                    #if word[0] not in ['있다', '했다', '한다', '없다', '된다']:
                    clean_words.append(word[0])
        return clean_words

# =========================================================================

embeddings_index = {}
f = codecs.open('/home/ubuntu/FastText/wiki.ko.vec', encoding='utf-8')

for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))

# ==================================================================================
from sklearn.utils import shuffle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

my_Tokenizer = MyTokenizer(Okt())

raw_docs_train = train_df['Comments'].tolist()
# raw_docs_test = test_df['document'].tolist()
# print(raw_docs_test)
num_classes = len(label_names)

processed_docs_train = []
# processed_docs_test = []

for doc in tqdm(raw_docs_train):
    tokens = my_Tokenizer(doc)
    processed_docs_train.append(tokens)

# for doc in tqdm(raw_docs_test):
#     tokens = my_Tokenizer(doc)
#     processed_docs_test.append(tokens)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
# tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)
tokenizer.fit_on_texts(processed_docs_train)
# tokenizer.fit_on_texts(processed_docs_train+proccessed_docs_test)
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
# word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size : ", len(word_index))

word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
# word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

y_train = train_df['label'].values
# ====================================================================================
# training params
batch_size = 256
num_epochs = 40

# model parameters
num_filters = 64
embed_dim = 300
weight_decay = 1e-4

# embedding matrix

words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+1)
embedding_matrix = np.zeros((nb_words, embed_dim))

# print(word_index)
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
        # cc.ko.300.vec에서 찾지 못한 단어들의 리스트.
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print("sample words not found: ", np.random.choice(words_not_found, 10))



879131it [02:05, 7020.04it/s]
  0%|          | 0/9615 [00:00<?, ?it/s]

found 879130 word vectors


100%|██████████| 9615/9615 [03:39<00:00, 43.75it/s]


dictionary size :  29113
number of null word embeddings: 9680
sample words not found:  ['학부성적' '딱이라는' '감싸는데서' '캐는게' '넘아' '엠비' '마시던데' '윌급' '챙기는거' '듣는거다']


In [60]:
# ========================================================================================
from keras.layers import BatchNormalization
import tensorflow as tf

model = tf.keras.Sequential()

model.add(Embedding(nb_words, embed_dim, input_length=max_seq_len, weights=[embedding_matrix], trainable=False))

model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))
model.summary()
# ================================================================================
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

es_callback = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(word_seq_train, y_train, batch_size=32,
          epochs=num_epochs, 
                    validation_split=0.2, 
                    callbacks=[es_callback], 
                    shuffle=False)

# predictions = model.predict_classes(word_seq_test)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 33, 300)           8734200   
_________________________________________________________________
dropout_8 (Dropout)          (None, 33, 300)           0         
_________________________________________________________________
dense_12 (Dense)             (None, 33, 32)            9632      
_________________________________________________________________
bidirectional_8 (Bidirection (None, 33, 128)           49664     
_________________________________________________________________
bidirectional_9 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_13 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)               

In [61]:
model.save('Naver_Comments_Model_All.h5')